<a href="https://colab.research.google.com/github/qwiksilva/cs224w-github-rec/blob/master/Github_Rec_FullData_GraphBuilder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [0]:
%reload_ext google.colab.data_table

In [0]:
# Fetch the largest 20 projects by the number of prs they have.
%%bigquery --project cs224w-257119 df
SELECT prs.num_prs, prs.base_repo_id, projects.name, projects.deleted
FROM (
  SELECT COUNT(base_repo_id) as num_prs, base_repo_id
  FROM `ghtorrent-bq.ght_2018_04_01.pull_requests`
  GROUP BY base_repo_id
) prs
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` projects ON prs.base_repo_id = projects.id
ORDER BY prs.num_prs DESC
LIMIT 20


In [0]:
df

,num_prs,base_repo_id,name,deleted
0,82846,20170979,everypolitician-data,False
1,69340,70604300,signcla-probe-repo,False
2,55760,789,liferay-portal,False
3,54805,33253535,sauron,False
4,40082,9570147,homebrew-cask,False
5,33504,6866209,homebrew,False
6,31212,202,gaia,False
7,30909,1803391,snoopys-sandbox,False
8,28820,3777,nixpkgs,False
9,28289,22091370,kubernetes,False


In [0]:
# Fetch the largest 20 projects by number of unqiue commenters
%%bigquery --project cs224w-257119 df
SELECT COUNT(comments.comment_id) as num_comments, COUNT(DISTINCT comments.user_id) as num_commenters, prs.base_repo_id
FROM `ghtorrent-bq.ght_2018_04_01.pull_request_comments` comments
INNER JOIN `ghtorrent-bq.ght_2018_04_01.pull_requests` prs ON prs.id = comments.pull_request_id
GROUP BY prs.base_repo_id
ORDER BY num_commenters DESC
LIMIT 20

In [0]:
%%bigquery --project cs224w-257119 pr_comments_spark
SELECT pr_comments.pull_request_id AS pr_id, 
pr.pullreq_id as github_pr_id,
pr_comments.user_id as commenter_id,
users1.login as commenter_username,
IFNULL(followers_stat.fl_count, 0) as commenter_follower_count,
IFNULL(commits_stat1.cm_count, 0) as commenter_total_github_commit_count, # As author
IFNULL(commits_stat2.cm_count, 0) as commenter_base_repo_commit_count, # As author
head_commits.author_id as head_commit_author_id,
users2.login as head_commit_author_username,
head_commits.committer_id as head_commit_committer_id, 
base_commits.author_id as base_commit_author_id,
users3.login as base_commit_author_username,
base_commits.committer_id as base_commit_committer_id,
pr.head_repo_id,
pr.base_repo_id,
base_project.owner_id as base_repo_owner,
users4.login as base_repo_owner_username,
head_project.owner_id as head_repo_owner_id,
users5.login as head_repo_owner_username,
pr.head_commit_id,
pr.base_commit_id,
pr.pullreq_id,
pr_comments.created_at as comment_created_at,
pr_comments.position as comment_position,
pr_comments.comment_id,
pr_comments.body as comment_body
FROM 
`ghtorrent-bq.ght_2018_04_01.pull_request_comments` pr_comments
INNER JOIN `ghtorrent-bq.ght_2018_04_01.pull_requests` pr
ON pr_comments.pull_request_id = pr.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.commits` head_commits
ON pr.head_commit_id = head_commits.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.commits` base_commits
ON pr.base_commit_id = base_commits.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` base_project
ON base_project.id = pr.base_repo_id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.projects` head_project
ON head_project.id = pr.head_repo_id
LEFT JOIN ( # Map with followers to count num-followers
    SELECT followers.follower_id, COUNT(DISTINCT followers.user_id) as fl_count
    FROM `ghtorrent-bq.ght_2018_04_01.followers` followers
    GROUP BY followers.follower_id
) followers_stat
ON pr_comments.user_id = followers_stat.follower_id
LEFT JOIN (  # Count num-contributions in entire Github
    SELECT commits1.author_id, COUNT(*) as cm_count
    FROM `ghtorrent-bq.ght_2018_04_01.commits` commits1
    GROUP BY commits1.author_id
) commits_stat1
ON pr_comments.user_id = commits_stat1.author_id
LEFT JOIN (  # Count num-contributions so far in the base repo
    SELECT commits2.author_id, commits2.project_id, COUNT(*) as cm_count
    FROM `ghtorrent-bq.ght_2018_04_01.commits` commits2
    GROUP BY commits2.author_id, commits2.project_id
) commits_stat2
ON pr_comments.user_id = commits_stat2.author_id AND pr.base_repo_id= commits_stat2.project_id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.users` users1 # Map all useful userid to username for sanity checking
ON pr_comments.user_id = users1.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.users` users2
ON head_commits.author_id = users2.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.users` users3
ON base_commits.author_id = users3.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.users` users4
ON base_project.owner_id = users4.id
INNER JOIN `ghtorrent-bq.ght_2018_04_01.users` users5
ON head_project.owner_id = users5.id
#Spark
WHERE pr.base_repo_id = 8196280

# Sanity Check

The head committer and author tends to be the same. The base committer and author can vary.
I checked, and the head commit ids match the user who submitted the pull request.
I'm still not sure how exactly to interpertet the base_commit_author and base_committer_ids. 
For rereference, this is the pullrequest corresponding to row 0 (github_pr_id 18476)

https://github.com/apache/spark/pull/18476

The usernames of the ids are as follows:

head_commit_author_id / head_commit_committer_id: **sadikovi**

base_commit_author_id: **liufengdb**

base_commit_committer_id: **zsxwing**


In [36]:
pr_comments_spark.loc[pr_comments_spark.pullreq_id == 18476, :]

,pr_id,github_pr_id,commenter_id,commenter_username,commenter_follower_count,commenter_total_github_commit_count,commenter_base_repo_commit_count,head_commit_author_id,head_commit_author_username,head_commit_committer_id,base_commit_author_id,base_commit_author_username,base_commit_committer_id,head_repo_id,base_repo_id,base_repo_owner,base_repo_owner_username,head_repo_owner_id,head_repo_owner_username,head_commit_id,base_commit_id,pullreq_id,comment_created_at,comment_position,comment_id,comment_body
60426,26035738,18476,4388120,sadikovi,20,2718,2,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 21:59:05+00:00,35,125137670,"Will do, thanks. I am going to update descript..."
60427,26035738,18476,4388120,sadikovi,20,2718,2,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 22:07:26+00:00,35,125138807,"Sure, will update accordingly. Thanks!"
60428,26035738,18476,4388120,sadikovi,20,2718,2,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 21:25:40+00:00,35,125133239,@tgravescs I updated comment to include note a...
60429,26035738,18476,6682,JoshRosen,401,6140,1782,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-07-01 00:19:33+00:00,31,125149417,If you're only documenting this in master then...
60430,26035738,18476,2062189,tgravescs,24,584,71,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 22:05:48+00:00,35,125138598,"ah sorry, my fault we should change ""value wil..."
60431,26035738,18476,2062189,tgravescs,24,584,71,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 13:28:37+00:00,35,125038588,we may want to add a comment about increasing ...
60432,26035738,18476,2062189,tgravescs,24,584,71,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 22:05:07+00:00,35,125138509,"yep, looks good."
60433,26035738,18476,2062189,tgravescs,24,584,71,4388120,sadikovi,4388120,10067869,liufengdb,478320,38647905,8196280,13369,apache,4388120,sadikovi,726517541,726415645,18476,2017-06-30 21:53:54+00:00,35,125137046,"thanks for updating, sorry a couple minor nits..."


In [35]:
pr_comments_spark.loc[pr_comments_spark.pullreq_id == 18476, ["commenter_username", "comment_created_at", "comment_body"]].values

array([['sadikovi', Timestamp('2017-06-30 21:59:05+0000', tz='UTC'),
        'Will do, thanks. I am going to update description to this:  ```  Capacity for event queue in Spark listener bus, must be greater than 0.   Consider increasing value (e.g. 20000) if listener events are dropped.   Increasing this value will result in the dr'],
       ['sadikovi', Timestamp('2017-06-30 22:07:26+0000', tz='UTC'),
        'Sure, will update accordingly. Thanks!'],
       ['sadikovi', Timestamp('2017-06-30 21:25:40+0000', tz='UTC'),
        '@tgravescs I updated comment to include note about memory increase. Could you review again, please? Thanks.'],
       ['JoshRosen', Timestamp('2017-07-01 00:19:33+0000', tz='UTC'),
        "If you're only documenting this in master then please use `spark.scheduler.listenerbus.eventqueue.capacity` instead (see definition in code and git blame for explanation)."],
       ['tgravescs', Timestamp('2017-06-30 22:05:48+0000', tz='UTC'),
        'ah sorry, my fault we

In [13]:
pr_comments_spark.loc[:, 'head_repo_owner'] # Within Spark multiple owner-id

0         6731670
1         6731670
2         6731670
3         6731670
4         6731670
           ...   
130629    4488100
130630    4488100
130631    4488100
130632    4488100
130633    4488100
Name: head_repo_owner, Length: 130634, dtype: int64

In [0]:
#Download to csv
from google.colab import files
path = './pr_comments_spark_with_commenter_stat_11012019.csv'
with open(path, 'w') as f:
  pr_comments_spark.to_csv(f)
files.download(path)

In [39]:
pr_comments_spark.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130634 entries, 0 to 130633
Data columns (total 26 columns):
pr_id                                  130634 non-null int64
github_pr_id                           130634 non-null int64
commenter_id                           130634 non-null int64
commenter_username                     130634 non-null object
commenter_follower_count               130634 non-null int64
commenter_total_github_commit_count    130634 non-null int64
commenter_base_repo_commit_count       130634 non-null int64
head_commit_author_id                  130634 non-null int64
head_commit_author_username            130634 non-null object
head_commit_committer_id               130634 non-null int64
base_commit_author_id                  130634 non-null int64
base_commit_author_username            130634 non-null object
base_commit_committer_id               130634 non-null int64
head_repo_id                           130634 non-null int64
base_repo_id                    